# HDDA. Home Assignment 56

This is your last home assignment. It consists of the last two homework assignments at once. Do not worry, both are quite simple. More than that it's well equipped with comments. It should be interesting to people who are fond of Data Science.

*Ok. Let's start.*

## Objective

Some of you have already heard about Skip-Gram Negative Sampling word embedding model. In other words it's word2vec. (*1.pdf is in repo*).

For the optimisation of SGNS word embedding model SGD is usually used. Since our course is based on the fundamental concepts of algebra,  an alternative way to solve the optimization problem has to be demostrated. Just because it is the power and beauty of mathematics.

Thus, the objective of this task is an optimization of SGNS formulated as implicit matrix factorization. (*2.pdf is in repo*)

###### 1. Basics
Assume we have a text corpus given as a sequence of words $\{w_1,w_2,\dots,w_n\}$ where $n$ may be larger than $10^{12}$ and $w_i \in \mathcal{V}$ belongs to a vocabulary of words $\mathcal{V}$. A word $c \in \mathcal{V}$ is called *a context* of word $w_i$ if they are found together in the text. More formally, given some measure $L$ of closeness between two words (typical choice is $L=2$), a word $c \in \mathcal{V}$ is called a context if $c \in \{w_{i-L}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+L} \}$ Let $\mathbf{w},\mathbf{c}\in\mathbb{R}^d$ be the *word embeddings* of word $w$ and context $c$, respectively. Assume they are specified by the mapping  $\Phi:\mathcal{V}\rightarrow\mathbb{R}^d$, so $\mathbf{w}=\Phi(w)$. The ultimate goal of SGNS word embedding model is to fit a good mapping $\Phi$.

Let $\mathcal{D}$ be a multiset of all word-contexts pairs observed in the corpus. In the SGNS model, the probability that word-context pair $(w,c)$ is observed in the corpus is modeled as the following distribution:

$$
P(\#(w,c)\neq 0|w,c) = \sigma(\mathbf{w}^\top \mathbf{c}) = \frac{1}{1 + \exp(-\mathbf{w}^\top \mathbf{c})},
$$

where $\#(w,c)$ is the number of times the pair $(w,c)$ appears in $\mathcal{D}$ and $\mathbf{w}^\top\mathbf{c}$ is the scalar product of vectors $\mathbf{w}$ and $\mathbf{c}$. Two important quantities which we will also use further are the number of times the word $w$ and the context $c$ appear in $\mathcal{D}$, which can be computed as

$$
\#(w) = \sum_{c\in\mathcal{V}} \#(w,c), \quad \#(c) = \sum_{w\in\mathcal{V}} \#(w,c).
$$


###### 2. Optimization objective

Vanilla word embedding models are trained by maximizing log-likelihood of observed word-context pairs, namely

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d}.
$$

Skip-Gram Negative Sampling approach modifies the objective by additionally minimizing the log-likelihood of random word-context pairs, so called *negative samples*. This idea incorporates some useful linguistic information that some number ($k$, usually $k=5$) of word-context pairs *are not* found together in the corpus which usually results in word embeddings of higher quality. The resulting optimization problem is

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + k \cdot \mathbb{E}_{c'\sim P_\mathcal{D}} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

where $P_\mathcal{D}(c)=\frac{\#(c)}{|\mathcal{D}|}$ is a probability distribution over word contexts from which negative samples are drawn.

*2.pdf* showed that this objective can be equivalently written as

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} f(w,c) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

A crucial observation is that this loss function depends only on the scalar product $\mathbf{w}^\top\mathbf{c}$ but not on embedding $\mathbf{w}$ and $\mathbf{c}$ separately.

###### 3. Matrix factorization problem statement

Let $|\mathcal{V}|=m$, $W \in \mathbb{R}^{m\times d}$ and $C \in \mathbb{R}^{m\times d}$ be matrices, where each row $\mathbf{w}\in\mathbb{R}^d$ of matrix $W$ is the word embedding of the corresponding word $w$ and each row $\mathbf{c}\in\mathbb{R}^d$ of matrix $C$ is the context embedding of the corresponding context $c$. SGNS embeds both words and their contexts into a low-dimensional space $\mathbb{R}^d$, resulting in the word and context matrices $W$ and $C$. The rows of matrix $W$ are typically used in NLP tasks (such as computing word similarities) while $C$ is ignored. It is nonetheless instructive to consider the product $W^\top C = M$. Viewed this way, SGNS can be described as factorizing an implicit matrix $M$ of dimensions $m \times m$ into two smaller matrices.

Which matrix is being factorized? A matrix entry $M_{wc}$ corresponds to the dot product $\mathbf{w}^\top\mathbf{c}$ . Thus, SGNS is factorizing a matrix in which each row corresponds to a word $w \in \mathcal{V}$ , each column corresponds to a context $c \in \mathcal{V}$, and each cell contains a quantity $f(w,c)$ reflecting the strength of association between that particular word-context pair. Such word-context association matrices are very common in the NLP and word-similarity literature. That said, the objective of SGNS does not explicitly state what this association metric is. What can we say about the association function $f(w,c)$? In other words, which matrix is SGNS factorizing? Below you will find the answers.

## Home assignment 5

#### This part is pure theoretical. 

Solve SGNS optimization problem with respect to the $\mathbf{w}^\top\mathbf{c}$ and show that the matrix being factorized is

$$
M_{wc} = \mathbf{w}^\top\mathbf{c} = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)
$$

**Hint:** Denote $x=\mathbf{w}^\top\mathbf{c}$, rewrite SGNG optimization problem in terms of $x$ and solve it.
                                                       
**Note:** This matrix is called Shifted Pointwise Mutual Information (SPMI) matrix, as its elements can be written as

$$
\text{SPMI}(w,c) = M_{wc} = \mathbf{w}^\top\mathbf{c} = \text{PMI}(w,c) - \log k
$$

and $\text{PMI}(w,c) = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{\#(w)\cdot\#(c)} \right)$ is the well-known *pointwise mutual information* of $(w,c)$.

### Solution (proof)

Local objective for a specific (w, c) pair:
$$
\mathcal{l}(w,c) = \#(w,c)\log\sigma(\mathbf{w}^\top\mathbf{c}) + k\cdot\#(w)\cdot\frac{\#(c)}{|\mathcal{D}|}\log\sigma(-\mathbf{w}^\top\mathbf{c})
$$
To optimize the objective, we define $x=\mathbf{w}^\top\mathbf{c}$ and find its partial derivative with respect to $x:$
$$\frac{\partial \mathcal{l}}{\partial x} = \#(w,c)\sigma(-x) + k\cdot\#(w)\cdot\frac{\#(c)}{|\mathcal{D}|}\log\sigma(x)$$
We compare the derivative to zero, and after some simplification, arrive at:
$$e^{2x} - \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} - 1 \right)e^x - \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right) = 0$$
If we define $y = e^x,$ this equation becomes a quadratic equation of $y$, which has two solutions,
$y = −1$ (which is invalid given the definition of $y$) and:
$$y = \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)}$$
Substituting $y$ with $e^x$ and $x$ with $\mathbf{w}^\top\mathbf{c}$ reveals:
$$\mathbf{w}^\top\mathbf{c} = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)
$$

## Home assignment 6

#### This part is pure practical.

In [1]:
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

I have prepared the dataset (*dataset.txt is in repo*) for you. It contains compressed Wikipedia articles. Let's have a look at data.

In [2]:
import re
file = open("./dataset.txt", "r")
doclist = [line for line in file]
docstr = ''.join(doclist)
sentences = re.split(r'[.!?]', docstr)
sentences = [sentence.split() for sentence in sentences if len(sentence) > 1]

In [3]:
print (sentences[1249])

['achilles', 'wrath', 'is', 'terrible', 'and', 'he', 'slays', 'many', 'trojan', 'warriors', 'and', 'allies', 'including', 'priam', 's', 'son', 'lycaon', 'whom', 'achilles', 'had', 'previously', 'captured', 'and', 'sold', 'into', 'slavery', 'but', 'who', 'had', 'been', 'returned', 'to', 'troy']


Construct the word vocabulary from the obtained sentences which enumerates words which occur more than $r=200$ times in the corpus.

In [4]:
from collections import Counter

def create_vocabulary(sentences, r=200):
    vocabulary = {}
    words = []
    for item in sentences:
        words.extend(item)
    count = Counter(words)
    i = 0
    for item in count:
        if count[str(item)]>=r and item not in vocabulary:
            vocabulary[item] = i 
            i += 1
    return vocabulary

In [5]:
vocab = create_vocabulary(sentences)

Scan the text corpus with sliding window of size $5$ and step $1$ (which corresponds to $L$=2) and construct co-occurrence word-context matrix $D$ with elements $D_{wc}=\#(w,c)$. Please, ignore words which occur less than $r=200$ times, but include them into the sliding window. Please, see the graphical illustration of the procedure described.

In [6]:
def create_corpus_matrix(sentences, vocabulary):
    n = len(vocabulary)
    corpus_matrix = np.zeros((n,n))
    for i in range(len(sentences)):
        for  j in range(len(sentences[i])):
            if sentences[i][j] in vocabulary:
                if max(0,j-2) == j-2:
                    for word in sentences[i][j-2:j]:
                        if word in vocabulary:
                            corpus_matrix[vocabulary[sentences[i][j]], vocabulary[word]] += 1
                        else:
                            continue
                else:
                    for word in sentences[i][0:j]:
                        if word in vocabulary:
                                corpus_matrix[vocabulary[sentences[i][j]], vocabulary[word]] += 1
                        else:
                            continue
                if min(j+3, len(sentences[i])) == j+3:
                    for word in sentences[i][j+1:j+3]:
                        if word in vocabulary:
                                corpus_matrix[vocabulary[sentences[i][j]], vocabulary[word]] += 1
                        else:
                            continue
                else:
                    for word in sentences[i][j+1:len(sentences[i])]:
                        if word in vocabulary:
                                corpus_matrix[vocabulary[sentences[i][j]], vocabulary[word]] += 1
                        else:
                            continue
            else:
                continue
                
    return corpus_matrix

In [7]:
D = create_corpus_matrix(sentences, vocab)

To find good word embeddings, *2.pdf* proposed to find rank-$d$ SVD of Shifted Positive Pointwise Mutual Information (SPPMI) matrix

$$
U \Sigma V^\top \approx \text{SPPMI},
$$

where $\text{SPPMI}(w, c) = \max\left(\text{SPMI}(w, c), 0 \right)$ and $\text{SPMI}(w, c)$ is the element of the matrix $\text{SPPMI}$ at position $(w, c)$.
Then use $W=U\sqrt{\Sigma}$ as word embedding matrix. Your task is to reproduce their results. Write function constructs $\text{SPPMI}$ matrix, computes its SVD and produces word-vectors matrix $W$. Pay attention that $\text{SPPMI}$ matrix is **sparse**!  

In [8]:
from scipy.sparse import csr_matrix

def compute_embeddings(D, k, d=200):
    rows = np.sum(D, axis = 1)[:, np.newaxis]
    clmn = np.sum(D, axis = 0)
    M = np.maximum(np.log(np.sum(D)/k*((D/rows)/clmn) ),0)
    U,S,V = svds(csr_matrix(M),d)
    embedding_matrix = U @ np.diag(np.sqrt(S))
    return embedding_matrix

In [9]:
k = 5 # negative sampling parameter
W = compute_embeddings(D, k)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


Write class **WordVectors** using provided template.

In [10]:
class WordVectors:
    
    def __init__(self, vocabulary, embedding_matrix):
        self.vocab = vocabulary
        self.W = embedding_matrix
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        
    def word_vector(self, word):
        """ 
        Takes word and returns its word vector.
        """
        word_vector = self.embedding_matrix[self.vocab[word]]
        return word_vector
    
    def nearest_words(self, word, top_n=10):
        """ 
        Takes word from the vocabulary and returns its top_n
        nearest neighbors in terms of cosine similarity.
        """
        cos = np.zeros(len(self.vocab))
        W_norm = self.W[self.vocab[word]] / np.linalg.norm(self.W[self.vocab[word]])
        for i in range(len(self.vocab)):
            cos[i] = np.dot(W_norm,self.W[i]) / np.linalg.norm(self.W[i])
        neighbors = [(self.inv_vocab[i], round(cos[i],3)) for i in np.argsort(cos)[-(top_n+1):-1]][::-1]
        return neighbors

In [11]:
model = WordVectors(vocab, W)

In [12]:
model.nearest_words("anarchism")

[('communism', 0.792),
 ('anarcho', 0.787),
 ('capitalism', 0.784),
 ('socialism', 0.752),
 ('liberalism', 0.727),
 ('criticisms', 0.705),
 ('capitalist', 0.662),
 ('fascism', 0.565),
 ('anarchist', 0.527),
 ('marxist', 0.518)]

In [13]:
model.nearest_words("rap")

[('hop', 0.828),
 ('hip', 0.817),
 ('funk', 0.758),
 ('rock', 0.737),
 ('punk', 0.706),
 ('music', 0.676),
 ('pop', 0.666),
 ('scene', 0.659),
 ('band', 0.657),
 ('jazz', 0.625)]

In [14]:
model.nearest_words("ussr")

[('ukraine', 0.671),
 ('russia', 0.629),
 ('poland', 0.55),
 ('belarus', 0.538),
 ('yugoslavia', 0.538),
 ('romania', 0.517),
 ('serbia', 0.507),
 ('austria', 0.5),
 ('hungary', 0.466),
 ('bulgaria', 0.43)]

Calculate top 10 nearest neighbours with the corresponding cosine similarities for the words {numerical, linear, algebra}

In [15]:
model.nearest_words("numerical")

[('computation', 0.547),
 ('mathematical', 0.532),
 ('calculations', 0.499),
 ('polynomial', 0.485),
 ('calculation', 0.473),
 ('practical', 0.46),
 ('statistical', 0.456),
 ('symbolic', 0.455),
 ('geometric', 0.441),
 ('simplest', 0.438)]

In [16]:
model.nearest_words("linear")

[('differential', 0.759),
 ('equations', 0.724),
 ('equation', 0.682),
 ('continuous', 0.674),
 ('multiplication', 0.674),
 ('integral', 0.672),
 ('algebraic', 0.667),
 ('vector', 0.654),
 ('algebra', 0.63),
 ('inverse', 0.622)]

In [17]:
model.nearest_words("algebra")

[('geometry', 0.795),
 ('calculus', 0.73),
 ('algebraic', 0.716),
 ('differential', 0.687),
 ('equations', 0.665),
 ('equation', 0.648),
 ('theorem', 0.647),
 ('topology', 0.634),
 ('linear', 0.63),
 ('integral', 0.618)]